In [1]:
import torch
import numpy as np
from utils.generate_csbm import *
from utils.train import *
from utils.utils import *
from models.models import *
from models.setup import *
import networkx as nx
import pickle

In [118]:
csbm1 = cSBM(N=40, d=25, mu=1, l=2, p=100)
G1 = nx.from_numpy_matrix(csbm1.A)
print (nx.is_connected(G1), csbm1.phi)
csbm1.generate_node_parameters()
csbm1.generate_node_data(120, "GC")

True 0.574124243317157


In [139]:
csbm2 = cSBM(N=40, d=5, mu=1, l=2, p=100)
G2 = nx.from_numpy_matrix(csbm2.A)
print (nx.is_connected(G2), csbm2.phi)
csbm2.generate_node_parameters()
csbm2.generate_node_data(120, "GC")

True 0.574124243317157


In [120]:
csbm3 = cSBM(N=40, d=15, mu=1, l=2, p=100)
G3 = nx.from_numpy_matrix(csbm3.A)
print (nx.is_connected(G3), csbm3.phi)
csbm3.generate_node_parameters()
csbm3.generate_node_data(120, "GC")

True 0.574124243317157


In [140]:
B = np.identity(40)/40 - np.ones((40,1))@np.ones((1,40))/(40**2)

In [141]:
L1 = np.array(nx.laplacian_matrix(G1).todense())
L2 = np.array(nx.laplacian_matrix(G2).todense())
L3 = np.array(nx.laplacian_matrix(G3).todense())

In [142]:
np.linalg.eigh(B@L1)[0].max()

0.8411162054440794

In [143]:
np.linalg.eigh(B@L2)[0].max()

0.26833169506315313

In [144]:
np.linalg.eigh(B@L3)[0].max()

0.5396689091255171

In [145]:
init_mlp = MLP(100, 64, 2, bias=False)

In [149]:
test_acc1 = []
for ith in range(20):
    csbm1.generate_node_data(120, "GC")
    A_tilde1 = calculate_Atilde(csbm1.A, K=10, alpha=0.9)
    server1 = set_up_GC(csbm1.Xs, csbm1.ys, init_mlp, A_tilde1, n_train=10, n_val=10, improved=False)
    tl, ta, vl, va = train_GC(server1, num_communication=150, batch_size=5, learning_rate=0.5, I=10,
                              gradient=True, noise=False, 
                              Print=False, print_time=10)
    test_acc1.append(server1.eval_test())
    print (ith)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [150]:
test_acc2 = []
for ith in range(20):
    csbm2.generate_node_data(120, "GC")
    A_tilde2 = calculate_Atilde(csbm2.A, K=10, alpha=0.9)
    server2 = set_up_GC(csbm2.Xs, csbm2.ys, init_mlp, A_tilde2, n_train=10, n_val=10, improved=False)
    tl2, ta2, vl2, va2 = train_GC(server2, num_communication=150, batch_size=5, learning_rate=0.5, I=10,
                              gradient=True, noise=False, 
                              Print=False, print_time=10)
    test_acc2.append(server2.eval_test())
    print (ith)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [151]:
test_acc3 = []
for ith in range(20):
    csbm3.generate_node_data(120, "GC")
    A_tilde3 = calculate_Atilde(csbm3.A, K=10, alpha=0.9)
    server3 = set_up_GC(csbm3.Xs, csbm3.ys, init_mlp, A_tilde3, n_train=10, n_val=10, improved=False)
    tl2, ta2, vl2, va2 = train_GC(server3, num_communication=150, batch_size=5, learning_rate=0.5, I=10,
                              gradient=True, noise=False, 
                              Print=False, print_time=10)
    test_acc3.append(server3.eval_test())
    print (ith)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [158]:
np.array(test_acc1)[,]

array([[0.1589472 , 0.93925   ],
       [0.15282577, 0.942     ],
       [0.15053613, 0.942     ],
       [0.16333588, 0.9345    ],
       [0.16856886, 0.938     ],
       [0.15396317, 0.942     ],
       [0.15575414, 0.9365    ],
       [0.15422784, 0.941     ],
       [0.15938819, 0.941     ],
       [0.16693176, 0.933     ],
       [0.14352908, 0.94875   ],
       [0.16766342, 0.9355    ],
       [0.16659309, 0.93425   ],
       [0.16662992, 0.93925   ],
       [0.14450309, 0.94175   ],
       [0.15609693, 0.94      ],
       [0.14432127, 0.94775   ],
       [0.15743623, 0.939     ],
       [0.17258136, 0.931     ],
       [0.16183076, 0.93875   ]])

In [ ]:
print (np.std(test_accs1), np.std(test_accs3), np.std(test_accs2))